In [14]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [15]:
# define the paths to all experiment data
path_group_0 = "/Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes"
path_group_1 = "/Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [16]:
FILE_PREFIX = "Enterobacter"
DATASET_OUT = "./01_GroupK/data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./01_GroupK/result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [17]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [18]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [19]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [20]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [21]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [22]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [23]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Define number of threads/core to use

In [ ]:
cnt_jobs = 40

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [24]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_group_0, path_group_1], ['d_E._aerogenes', 'e_E._cloacae'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/d_E._aerogenes
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae
root - INFO - Loading files from /Users/Praktikum/Downloads/DATA_tsv_aggregated_2/Ascites/Gram_-/Enterobacteriacea/Enterobacter/e_E._cloacae


ValueError: invalid literal for int() with base 10: 'E'

# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [ ]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

# Preprocess the data

In [ ]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

# Implement GroupKFold CV
## LDA Dimensionality Reduction 

In [ ]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()



### Baseline with LDA alone

In [ ]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=logo, scoring=SCORING, return_train_score=True, groups=groups, n_jobs=cnt_jobs)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)
result

### Feature Selection with PCA followed by LDA

In [ ]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


### Non-Negative Matrix Factorization

In [ ]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)"""

### Feature Agglomeration

In [ ]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### PeakPicker

In [ ]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


## Regularized Models
### Logistic Regression L1 Penalty

In [ ]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### Logistic Regression L2 Penalty

In [ ]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### Linear SVM L1 Penalty

In [ ]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=10000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### Linear SVM L2 Penalty

In [ ]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

## Tree-based models
### Basic Decision Tree


In [ ]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### Random Forest

In [ ]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

### Gradient-boosted Decision Tree

In [ ]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3, n_jobs=cnt_jobs).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)